In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from airsea.windstress import stress
from matplotlib import mlab
import scipy.signal as signal
import scipy.stats as st
from scipy.interpolate import interp1d
from scipy import arange, array, exp, integrate
from physoce import tseries as ts
import gsw

def neureg(x,y):
    #x,y are numpy arrays
    #subtract mean to make zero mean:
    xn = x - np.mean(x)
    yn = y - np.mean(y)
    #perform neutral regression for comparison, e.g. Garrett and Petrie 1981 and Kirincich et al 2005
    an = np.sqrt(((st.tvar(yn)))/((st.tvar(xn)))) #find slope
    bn = (np.mean(y) - an*np.mean(x)) #find intercept
    rn = np.sum(x*y)/np.sqrt((np.sum(x**2)*np.sum(y**2))) #find correlation coefficient
    slope_err = an*np.sqrt((1-rn**2)/len(x)) #compute std err in slope = a*sqrt((1-r^2)/n)
    #compute std err in intercept = vary^.5*sqrt((1-r^2)/n*(1+xmean^2/var(x))) 
    #(e.g., Miller & Kahn 1962, Statistical Analysis in the Geological Science)
    #https://stats.stackexchange.com/questions/391112/standard-error-of-coefficient-estimates-for-model-ii-regression
    inter_err = np.sqrt(st.tvar(yn))*np.sqrt(((1-rn**2)/len(x))*(1+np.mean(x)**2/st.tvar(xn))) 
    return an,bn,rn,slope_err,inter_err

def extrap1d(interpolator):
    xs = interpolator.x
    ys = interpolator.y

    def pointwise(x):
        if x < xs[0]:
            return ys[0]+(x-xs[0])*(ys[1]-ys[0])/(xs[1]-xs[0])
        elif x > xs[-1]:
            return ys[-1]+(x-xs[-1])*(ys[-1]-ys[-2])/(xs[-1]-xs[-2])
        else:
            return interpolator(x)

    def ufunclike(xs):
        return np.array(list(map(pointwise, np.array(xs))))

    return ufunclike

def rot(u,v,theta):
    w = u + 1j*v
    ang = theta*np.pi/180
    wr = w*np.exp(1j*ang)
    ur = np.real(wr)
    vr = np.imag(wr)
    return ur,vr

def princax(u,v=None):

    # if one input only, decompose complex vector
    if v is None:
        w = np.copy(u)
        u = np.real(w)
        v = np.imag(w)

    # only use finite values for covariance matrix
    ii = np.isfinite(u+v)
    uf = u[ii]
    vf = v[ii]

    # compute covariance matrix
    C = np.cov(uf,vf)

    # calculate principal axis angle (ET, Equation 4.3.23b)
    theta = 0.5*np.arctan2(2.*C[0,1],(C[0,0] - C[1,1])) * 180/np.pi

    # calculate variance along major and minor axes (Equation 4.3.24)
    term1 = C[0,0] + C[1,1]
    term2 = ((C[0,0] - C[1,1])**2 + 4*(C[0,1]**2))**0.5
    major = np.sqrt(0.5*(term1 + term2))
    minor = np.sqrt(0.5*(term1 - term2))

    return theta,major,minor

def wind_uv_from_spddir(wspd,wdir):
    theta = np.array(wdir)
    theta = theta*np.pi/180
    x = np.sin(theta)
    y = np.cos(theta)
    theta_cart = np.arctan2(y,x)
    u = -wspd*np.cos(theta_cart) #why should these be negative?
    v = -wspd*np.sin(theta_cart)
    return u,v

C:\Users\asche\Anaconda3\envs\reu21\lib\site-packages\airsea-0.0.1-py3.9.egg\airsea\atmosphere.py:363: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\asche\Anaconda3\envs\reu21\lib\site-packages\airsea-0.0.1-py3.9.egg\airsea\atmosphere.py:365: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\asche\Anaconda3\envs\reu21\lib\site-packages\airsea-0.0.1-py3.9.egg\airsea\atmosphere.py:363: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\asche\Anaconda3\envs\reu21\lib\site-packages\airsea-0.0.1-py3.9.egg\airsea\atmosphere.py:365: SyntaxWarning: "is" with a literal. Did you mean "=="?


# Data Retrieval and Preparation

## Load in datasets for wind/velocity

In [2]:
ncfile = 'https://dods.ndbc.noaa.gov/thredds/dodsC/data/stdmet/nwpo3/nwpo3h2019.nc'
ds_wind = xr.open_dataset(ncfile)
ds_wind.load()

<xarray.Dataset>
Dimensions:                  (latitude: 1, longitude: 1, time: 8738)
Coordinates:
  * time                     (time) datetime64[ns] 2019-01-01T01:00:00 ... 20...
  * latitude                 (latitude) float32 44.61
  * longitude                (longitude) float32 -124.1
Data variables: (12/13)
    wind_dir                 (time, latitude, longitude) float64 81.0 ... 190.0
    wind_spd                 (time, latitude, longitude) float32 2.5 ... 14.1
    gust                     (time, latitude, longitude) float32 2.6 ... 17.5
    wave_height              (time, latitude, longitude) float32 nan nan ... nan
    dominant_wpd             (time, latitude, longitude) timedelta64[ns] NaT ...
    average_wpd              (time, latitude, longitude) timedelta64[ns] NaT ...
    ...                       ...
    air_pressure             (time, latitude, longitude) float32 1.03e+03 ......
    air_temperature          (time, latitude, longitude) float32 7.1 4.9 ... 9.9
    sea_surface_temperature  (time, latitude, longitude) float32 nan nan ... nan
    dewpt_temperature        (time, latitude, longitude) float32 nan nan ... nan
    visibility               (time, latitude, longitude) float32 nan nan ... nan
    water_level              (time, latitude, longitude) float32 nan nan ... nan
Attributes:
    institution:                     NOAA National Data Buoy Center and Parti...
    url:                             http://dods.ndbc.noaa.gov
    quality:                         Automated QC checks with manual editing ...
    conventions:                     COARDS
    station:                         nwpo3
    comment:                         Newport, OR
    location:                        44.613 N 124.067 W 
    DODS_EXTRA.Unlimited_Dimension:  time

In [3]:
#use combined dataset from other analysis for convenience!
ncfile = 'C:/Users/asche/REU21/netcdf/inshore_surface_adcp_comb_linear.nc'
ds_velo = xr.open_dataset(ncfile)
ds_velo.load()

<xarray.Dataset>
Dimensions:           (bin_depths_avg: 22, time: 2235)
Coordinates:
  * time              (time) datetime64[ns] 2019-04-20T03:30:00 ... 2019-10-2...
  * bin_depths_avg    (bin_depths_avg) float64 -21.98 -20.98 ... -1.98 -0.9797
Data variables: (12/17)
    u                 (bin_depths_avg, time) float64 0.0781 0.1144 ... 1.195
    v                 (bin_depths_avg, time) float64 0.08675 0.1102 ... -0.2696
    sea_surface       (time) float64 -1.069 -0.4755 0.8208 ... 0.5014 0.2247
    bottom_depth_avg  float64 24.08
    u_fil             (bin_depths_avg, time) float64 nan nan nan ... nan nan nan
    v_fil             (bin_depths_avg, time) float64 nan nan nan ... nan nan nan
    ...                ...
    as_fil            (bin_depths_avg, time) float64 nan nan nan ... nan nan nan
    theta             float64 80.67
    cs_bar            (bin_depths_avg, time) float64 nan nan nan ... nan nan nan
    as_bar            (bin_depths_avg, time) float64 nan nan nan ... nan nan nan
    us_comb           (time) float64 nan nan nan nan nan ... nan nan nan nan nan
    zs_comb           (time) float64 nan nan nan nan nan ... nan nan nan nan nan

## Get velocity and stress components from wind

In [4]:
ds_wind = ds_wind.squeeze()

#get velocity components
ds_wind['wind_east'],ds_wind['wind_north'] = wind_uv_from_spddir(ds_wind['wind_spd'], ds_wind['wind_dir'])

#get wind stress
tau = stress(ds_wind['wind_spd'],z=4.1)
tau_east = tau*np.cos(np.arctan2(ds_wind['wind_north'],ds_wind['wind_east']))
tau_north = tau*np.sin(np.arctan2(ds_wind['wind_north'],ds_wind['wind_east']))

ds_wind['tau_east'] = (('time'), tau_east)
ds_wind['tau_north'] = (('time'), tau_north)

## Rotate wind vectors

In [ ]:
ds_wind['tau_x'],ds_wind['tau_y'] = rot(ds_wind['tau_east'],ds_wind['tau_north'],-ds_velo['theta']+90)
#plot to make sure it is reasonable
plt.figure()
plt.plot(ds_wind['tau_east'],ds_wind['tau_north'], '.', label='unrotated')
plt.plot(ds_wind['tau_x'],ds_wind['tau_y'],'.', label='rotated')
plt.legend()
plt.axis('equal')

## Bring both adcp and wind data onto common dimension of time

In [ ]:
ds_velo = ds_velo.resample(time='1h').mean()

In [ ]:
ds_wind = ds_wind.resample(time='1h').mean()

In [ ]:
t1 = np.datetime64('2019-05-01 00:00:00')
t2 = np.datetime64('2019-09-30 00:00:00')
good_time_velo = (ds_velo['time'] >= t1) & (ds_velo['time'] < t2)
good_time_wind = (ds_wind['time'] >= t1) & (ds_wind['time'] < t2)

In [ ]:
wind_ekman = ds_wind['tau_y'].where(good_time_wind)/(1025*gsw.f(ds_windresa['latitude'].where(good_time_wind).values))

In [ ]:
mask_wind = (wind_ekman['time'] >= t1) & (wind_ekman['time'] < t2)
wind_ekman_mask = wind_ekman[mask_wind]

## Extract sea surfact transport and surface layer depth from velocity dataset

In [ ]:
us, zs = ds_velo['us_comb'].where(good_time_velo), ds_velo['zs_comb'].where(good_time_velo)

In [ ]:
mask_velo = (us['time'] >= t1) & (us['time'] < t2)
us_mask = us[mask_velo]

In [ ]:
us = us.astype(float)

In [ ]:
us[us == 0] = np.nan #convert zeros in array to nan so linregress function works

# Trasport Fraction Analysis

## Least Squares Linear Regression

In [ ]:
mask = ~np.isnan(wind_ekman_mask) & ~np.isnan(us_mask) & mask_wind & mask_velo #mask all values with nan in them

In [ ]:
result = st.linregress(wind_ekman_mask[mask],us_mask[mask])

In [ ]:
#give results names for convenience
a = result.slope
b = result.intercept
astd = result.stderr
bstd = result.intercept_stderr

In [ ]:
#plot results from linear least squares regression
plt.plot(figsize=(15,4))
plt.plot(wind_ekman_mask,us_mask,'.')
plt.plot(wind_ekman_mask,wind_ekman_mask*a+b)
plt.xlabel('Wind-Derived Ekman Transport [$m^2/s$]')
plt.ylabel('Observational Ekman Transport [$m^2/s$]')
#plt.axis('equal')
textstr = 'a = {:.4f} $\pm$ {:.4f}\nb= {:.4f} $\pm$ {:.4f}'.format(a,1.96*astd,b,1.96*bstd)
props = dict(boxstyle='round', facecolor='w')
plt.text(3, -0.4, textstr, fontsize=10,
        verticalalignment='top', bbox=props)

Results from linear regression are shown with 95% confidence interval given as 1.96\*stderr

## Neutral Regression

In [ ]:
an,bn,rn,anstd,bnstd = neureg(wind_ekman_mask[mask].values,us_mask[mask].values)

In [ ]:
#plot neutral regression fits
plt.figure()
plt.plot(wind_ekman_mask,us_mask,'.')
plt.plot(wind_ekman_mask,wind_ekman_mask*an+bn)
textstr = 'a = {:.4f} $\pm$ {:.4f}\nb= {:.4f} $\pm$ {:.4f}'.format(an,1.96*anstd,bn,1.96*bnstd)
props = dict(boxstyle='round', facecolor='w')
plt.text(3, -0.6, textstr, fontsize=10,
        verticalalignment='top', bbox=props)
#plt.ylim([-1,1])
plt.xlabel('Wind-Derived Ekman Transport [$m^2/s$]')
plt.ylabel('Observational Ekman Transport [$m^2/s$]')